##### Gerador : Recebe entradas aleatórias e tenta gerar algo parecido com o treinamento
##### Discriminador : Coleta dados de treinamento e dados gerados e tenta distinguir os falsos dos verdadeiros.

###### resumo -> gerador gera dados falsos e discriminador melhor reconhecimento de dados falsos

In [4]:
import numpy as np
import math 

### Funções

In [1]:
#transforma um número em uma lista de binários
def create_binary_list_from_int(num):
    if num < 0 or type(num) is not int:
        raise ValueError('Apenas inteiros positivos permitidos')
    return [int(x) for x in list(bin(num))[2:]]

In [2]:
#Gera uma lista de tuplas de binários 
def generate_even_data(max_int,batch_size = 16):
    #Número de binarios necessários para o maximo número
    max_length = int(math.log(max_int,2))
    
    #Gera os inteiros pares
    sampled_integers = np.random.randint(0,int(max_int/2),batch_size)
    
    #Gera os labels com valor 1(par)
    labels = [1]*batch_size
    
    #Gera a lista de números binários para o treino
    data = [create_binary_list_from_int(int(x*2)) for x in sampled_integers]
    
    #Padroniza o tamanho do vetor colocando zeros à esquerda
    data = [([0] * (max_length - len(x))) + x for x in data ]
    
    return labels,data
    

In [3]:
import torch.nn as nn

### Gerador e Discriminador

In [5]:
class Generator(nn.Module):
    
    def __init__(self,input_length):
        super(Generator,self).__init__()
        self.dense_layer = nn.Linear(int(input_length),int(input_length))
        self.activation = nn.Sigmoid()
        
    def forward(self,x):
        return self.activation(self.dense_layer(x))

In [6]:
class Discriminator(nn.Module):
    def __init__(self,input_length):
        super(Discriminator,self).__init__()
        self.dense = nn.Linear(int(input_length),1);
        self.activation = nn.Sigmoid()
        
    def forward(self,x):
        return self.activation(self.dense(x))

In [7]:
import math
import torch

### Treinamento

In [8]:
def train(max_int = 128,batch_size = 16,training_steps = 500):
    input_length=int(math.log(max_int,2))

    #Modelss

    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    #Otimizadores
    generator_optimizer = torch.optim.Adam(generator.parameters(),lr = 0.001)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(),lr = 0.001)

    #loss
    loss = nn.BCELoss()

    for i in range(training_steps):
        #Zera os gradientes
        generator_optimizer.zero_grad()
    
        #ruído na entrada do gerador do tipo float
    
        noise = torch.randint(0,2,size=(batch_size,input_length)).float()
        generated_data = generator(noise)
        
        #Gera exemplos de dados reais
        true_labels,true_data = generate_even_data(max_int,batch_size = batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()
    

        #Treina o gerador
        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out,true_labels)
        generator_loss.backward()
        generator_optimizer.step()
        
    
        #Treina o discriminador no true/generated data
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out,true_labels)
    
        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(generator_discriminator_out,torch.zeros(batch_size))
        discriminator_loss = (true_discriminator_loss+generator_discriminator_loss)/2
    
        discriminator_loss.backward()
    
        discriminator_optimizer.step()





        

